In [117]:
import numpy as np
from models.base_models import Transformer
from arithmetic_sampler import ArithmeticSampler
from config import get_config
from train import train
import torch
import torch.nn as nn
import torch.nn.functional as F
import utils as u

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
%%timeit
sampler.generate(2**16)

1.17 s ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [168]:
config = get_config()
sampler = ArithmeticSampler(config.task.max_variables)

In [169]:
model = Transformer(config)

train(model, sampler, config, verbose=False)

Results are saved in:  results\train_530aa16d998938de9243667df37bede9
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Transformer                              [256, 82]                 [256, 82, 16]             --
├─Embedding: 1-1                         [256, 82]                 [256, 82, 64]             1,024
├─ModuleList: 1-2                        --                        --                        --
│    └─TFBlock: 2-1                      [256, 82, 64]             [256, 82, 64]             --
│    │    └─LayerNorm: 3-1               [256, 82, 64]             [256, 82, 64]             128
│    │    └─MultiHeadAttention: 3-2      [256, 82, 64]             [256, 82, 64]             16,384
│    │    └─Dropout: 3-3                 [256, 82, 64]             [256, 82, 64]             --
│    │    └─LayerNorm: 3-4               [256, 82, 64]             [256, 82, 64]             128
│    │    └─Sequential: 3-5              [256, 82, 6

eval/accuracy,▁▂▃▄▄▆▆▆▇▇▇▇▇▇▇█████████████████████████
eval/loss,█▅▄▄▄▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,██▇▅▅▄▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇████▇▇▇▆▆▅▅▄▄▄▄▃▃▃▃▂▂▂▁▁
eval/accuracy,0.93138
eval/loss,0.7169
train/loss,0.75416
train/lr,0


Start training...


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

...

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

Training complete.


In [174]:
batch, mask = sampler.generate(1)
sampler.decode(batch)

['                (1+(((1-8)+(5-(3-1)))-(2-1)))+(6+5)=(1+((3+(5-(3-1)))-(2-1)))+(6+5)']

In [175]:
batch = batch.to(config.device)
logits, _ = model(batch)
(logits[:, :-1].argmax(dim=-1) - batch[:, 1:])[mask[:,1:]]

tensor([0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [178]:
sampler.decode(logits[:, :-1].argmax(dim=-1)[mask[:,1:]])

['(1+((9+(5-(3-1)))-(2-1)))+(6+5)']

In [179]:
sampler.decode(batch)

['                (1+(((1-8)+(5-(3-1)))-(2-1)))+(6+5)=(1+((3+(5-(3-1)))-(2-1)))+(6+5)']